## Importing the Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

In [2]:
PROJECT_PATH = os.path.join(os.getcwd(), 'my-own-face-detector-project')

In [3]:
DATASET_PATH = os.path.join(PROJECT_PATH, 'datasets')

## Augmenting the Training Images

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    '/Users/devrrior/Repos/my_lab_ai/notebooks/my-own-face-detector-project/datasets',
    target_size=(224,224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)


Found 265 images belonging to 2 classes.


In [5]:
train_generator.class_indices.values()
# dict_values([0, 1, 2])
NO_CLASSES = len(train_generator.class_indices.values())

## Building the Model

In [6]:
from keras_vggface.vggface import VGGFace

base_model = VGGFace(include_top=False,
model='vgg16',
input_shape=(224, 224, 3))
base_model.summary()
print(len(base_model.layers))
# 19 layers after excluding the last few layers


Model: "vggface_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 56, 56, 128)     

In [7]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)

# final layer with softmax activation
preds = Dense(NO_CLASSES, activation='softmax')(x)

In [8]:
# create a new model with the base model's original input and the 
# new model's output
model = Model(inputs = base_model.input, outputs = preds)
model.summary()

# don't train the first 19 layers - 0..18
for layer in model.layers[:19]:
    layer.trainable = False

# train the rest of the layers - 19 onwards
for layer in model.layers[19:]:
    layer.trainable = True


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 56, 56, 128)       0     

In [9]:
model.compile(optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [12]:
model.fit(train_generator,
  batch_size = 1,
  verbose = 1,
  epochs = 8)

Epoch 1/8
9/9 [==============================] - 36s 4s/step - loss: 4.4985e-10 - accuracy: 1.0000
Epoch 2/8
9/9 [==============================] - 37s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/8
9/9 [==============================] - 38s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/8
9/9 [==============================] - 41s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/8
9/9 [==============================] - 44s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/8
9/9 [==============================] - 45s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/8
9/9 [==============================] - 47s 6s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/8
9/9 [==============================] - 49s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000


In [ ]:
# creates a HDF5 file
model.save(
    './face_cnn_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# deletes the existing model
del model

# returns a compiled model identical to the previous one
model = load_model(

    './face_cnn_model.h5')


In [ ]:
import pickle

class_dictionary = train_generator.class_indices
class_dictionary = {
    value:key for key, value in class_dictionary.items()
}
print(class_dictionary)


In [ ]:
# save the class dictionary to pickle
face_label_filename = 'face-labels.pickle'
with open(face_label_filename, 'wb') as f: pickle.dump(class_dictionary, f)
import cv2
import os
import pickle
import numpy as np
import pickle

from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras_vggface import utils

# dimension of images
image_width = 224
image_height = 224

# load the training labels
face_label_filename = 'face-labels.pickle'
with open(face_label_filename, "rb") as \
    f: class_dictionary = pickle.load(f)

class_list = [value for _, value in class_dictionary.items()]
print(class_list)
import cv2
import os
import pickle
import numpy as np
import pickle

from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras_vggface import utils

# dimension of images
image_width = 224
image_height = 224

# load the training labels
face_label_filename = 'face-labels.pickle'
with open(face_label_filename, "rb") as \
    f: class_dictionary = pickle.load(f)

class_list = [value for _, value in class_dictionary.items()]
print(class_list)


In [ ]:
import cv2
import os
import pickle
import numpy as np
import pickle

from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras_vggface import utils

# dimension of images
image_width = 224
image_height = 224

# load the training labels
face_label_filename = 'face-labels.pickle'
with open(face_label_filename, "rb") as f:
    class_dictionary = pickle.load(f)

class_list = [value for _, value in class_dictionary.items()]
print(class_list)


In [ ]:
import mtcnn
face_detector = mtcnn.MTCNN()

In [ ]:
def load_images(path, width, height):
    image_files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

    processed_images = []

    for image_file in image_files:
        if image_file.endswith('.jpg') or image_file.endswith('.jpeg'):
            image_path = os.path.join(path, image_file)
            print(image_path)
            image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
            img_roi = face_detector.detect_faces(image)
    
            if img_roi:
                x1, y1, width, height = img_roi[0]["box"]
                x2, y2 = x1 + width, y1 + height
                face = image[y1:y2, x1:x2]
                face = cv2.resize(face, (224,224))
    
                processed_images.append(face)
                print(face.shape)
                plt.imshow(face)
                plt.show()

    processed_images_array = np.array(processed_images)

    return processed_images_array

In [ ]:
samples1 = load_images('/Users/devrrior/Repos/my_lab_ai/notebooks/my-own-face-detector-project/test', 224, 224)

In [ ]:
predicted_prob = model.predict(samples1)
print(predicted_prob)
print(predicted_prob[0].argmax())
print("Predicted face: " + class_list[predicted_prob[3].argmax()])
print("============================\n")